# Workflow of InSAR time series analysis (`pysarApp.py`)

We have implemented a generic routine processing workflow for InSAR time series analysis from a stack of unwrapped interferograms to displacement time-series. The workflow consists of two main blocks:   
1. correcting unwrapping errors and inversion for the raw phase time-series (blue ovals),   
2. correcting for noise from different sources to obtain the displacement time-series (green ovals).    

It includes some optional steps, which are switched off by default (marked by dashed boundaries), here we present the workflow in its most complete form. Configuration parameters for each step are initiated with default values in a customizable text file: [pysarApp_template.txt](https://github.com/yunjunz/PySAR/blob/master/docs/pysarApp_template.txt).     

<p align="left">
  <img width="600" src="docs/figs/workflow.png">
</p>     

## 1. Start point: InSAR stack (`load_data.py`)

The starting point is a stack of phase-unwrapped interferograms coregistered to a common SAR acquisition, corrected for earth curvature and topography. PySAR currently supports interferogram stacks produced by ROI_PAC, ISCE and GAMMA software (Rosen et al., 2004; Rosen et al, 2012; Werner et al., 2000). Some example file/directory structures are shown below:

- [ISCE/topsStack](docs/dir_isce_tops.ipynb)
- [ISCE/stripmapStack](docs/dir_isce_stripmap.ipynb)
- [ROI_PAC](docs/dir_roipac.ipynb)
- [GAMMA](docs/dir_gamma.ipynb)

### Example InSAR stack: [FernandinaSenD128](https://zenodo.org/record/2562349)

- Sensor: Sentinel-1 A/B descending track 128<br>
- Time: 13 Dec 2014 to 19 Jun 2018, 98 SAR acquisitions<br>
- Processor: ISCE/topsStack (Rosen et al, 2012; Fattahi et al, 2016)<br>      

Here we use a stack of interferograms from Sentinel-1 satellite acquired over Fernandina volcano, Galapagos, Ecuador as an example. We pair each SAR image with its three nearest neighbors back in time (sequential network; 288 interferograms in total); we multilook each interferogram by 15 and 5 looks in range and azimuth direction respectively, filter using a Goldstein filter with a strength of 0.2. We remove the topographic phase component using SRTM DEM (SRTMGL1, ~30m, 1 arc second with void-filled; Farr et al., 2005). The interferograms are phase-unwrapped using the minimum cost flow method (SNAPHU, Chen and Zebker, 2001).

In [2]:
import os
work_dir = os.path.expanduser('~/Work')
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print('Create directory: {}'.format(work_dir))
os.chdir(work_dir)
print('Go to work directory: {}'.format(work_dir))

if not os.path.isfile('FernandinaSenDT128.tar.xz'):
    !wget https://zenodo.org/record/2562349/files/FernandinaSenDT128.tar.xz
    !tar -xvJf FernandinaSenDT128.tar.xz
proj_dir = os.path.join(work_dir, 'FernandinaSenDT128/PYSAR')
os.chdir(proj_dir)
print('Go to project directory: {}'.format(proj_dir))

Go to work directory: /Users/yunjunz/Work
Go to project directory: /Users/yunjunz/Work/FernandinaSenDT128/PYSAR


In [7]:
!pysarApp.py FernandinaSenDT128.txt -g                                 #short custom template -> full default template
!load_data.py -t pysarApp_template.txt --project FernandinaSenDT128    #load binary files to HDF5 files


_________________________________________________
       ____             __     __     ____  
       /    )         /    )   / |    /    )
------/____/----------\-------/__|---/___ /------
     /        /   /    \     /   |  /    |  
____/________(___/_(____/___/____|_/_____|_______
                /                           
            (_ /                            

 A Python package for InSAR time series analysis.
           PySAR v1.0.0-dev, 2019-01-24
_________________________________________________

Project name: FernandinaSenDT128
Go to work directory: /Users/yunjunz/Work/FernandinaSenDT128/PYSAR

**********  Read Template File  **********
latest template file detected: /Users/yunjunz/Work/FernandinaSenDT128/PYSAR/pysarApp_template.txt
['/Users/yunjunz/Work/FernandinaSenDT128/PYSAR/INPUTS/FernandinaSenDT128.txt'] exists and is newer than ['/Users/yunjunz/Work/FernandinaSenDT128/PYSAR/FernandinaSenDT128.txt'], skip updating.
read custom template file: /Users/yunjunz/Work/F

In [12]:
!ls -l INPUTS                      #check loaded files in PYSAR/INPUTS directory
!info.py INPUTS/ifgramStack.h5     #use info.py / gdalinfo to check the HDF5 file information

total 1643440
-rw-r--r--  1 yunjunz  staff  109292360 Feb  5 21:54 ECMWF.h5
-rw-r--r--  1 yunjunz  staff       1872 Feb  5 22:13 FernandinaSenDT128.txt
-rw-r--r--  1 yunjunz  staff    2808168 Feb 12 00:06 geometryRadar.h5
-rw-r--r--  1 yunjunz  staff  729332240 Feb 12 00:06 ifgramStack.h5
******************** Basic File Info ************************
file name: /Users/yunjunz/Work/FernandinaSenDT128/PYSAR/INPUTS/ifgramStack.h5
file type: ifgramStack
coordinates : RADAR

******************** HDF5 File Structure ********************
Attributes in / level:
  ALOOKS                   1
  ANTENNA_SIDE             -1
  AZIMUTH_PIXEL_SIZE       15.616425207715931
  CENTER_LINE_UTC          42584.0
  DATA_TYPE                FLOAT
  DATE12                   141213-141225
  EARTH_RADIUS             6337286.638938101
  FILE_LENGTH              450
  FILE_PATH                /Users/yunjunz/Work/FernandinaSenDT128/ISCE/merged/interferograms/20141213_20141225/filt_fine.unw
  FILE_TYPE               

### 1.1 Select reference point (`reference_point.py`)


